Contenido bajo licencia Creative Commons BY-NC-ND. El contenido de este Notebook es creado por Ana Beatriz Acevedo Jaramillo y Daniela González González 2020. Este material es parte del curso Análisis de Estructuras del programa de Ingeniería Civil de la Universidad EAFIT.

<img src= 
"http://www.eafit.edu.co/biblioteca/PublishingImages/logos/eafit_apa.jpg" 
         alt="GeeksforGeeks logo" 
         align="left">

# *Análisis matricial de pórticos planos*

<p style='text-align: justify;'> Este Notebook fue desarrollado como parte del material de apoyo para realizar el análisis de pórticos planos. Debe tener en cuenta que los datos de entrada que aparecen por defecto corresponden al ejercicio explicado en el documento que acompaña a este Notebook.</p>

## DATOS DE ENTRADA

<p style='text-align: justify;'> Esta sección contiene los datos que debe ingresar el usuario para que el Notebook realice en ánalisis de la estructura.</p>

In [1]:
import numpy as np
import pandas as pd

### Información general de la estructura: 

In [2]:
Nn=4 #Número de nodos    
Ne=3 #Número de elementos   
NGLL=6 #Número de grados de libertad libres

Con la información anterior se cálcula lo siguiente:

In [3]:
NGL=3*Nn #Cálculo del número de grados de libertad
NGLR=NGL-NGLL #Cálculo del número de grados de libertad restringidos

### Grados de libertad asociados a cada elemento:


$$
\begin{align}
MGL = \\
\end{align} 
\begin{bmatrix}
gxi_{0} &gyi_{0} &gzi_{0} &gxf_{0} &gyf_{0} &gzf_{0}\\
gxi_{1} &gyi_{1} &gzi_{1} &gxf_{1} &gyf_{1} &gzf_{1}\\
gxi_{2} &gyi_{2} &gzi_{2} &gxf_{2} &gyf_{2} &gzf_{2}\\
. &. &. &. &. &.\\
gxi_{n} &gyi_{n} &gzi_{n} &gxf_{n} &gyf_{n} &gzf_{n}\end{bmatrix}\, .$$


In [4]:
MGL=[[6,7,8,0,1,2],[0,1,2,3,4,5],[3,4,5,9,10,11]] #Matriz con información de los grados de libertad asociados a cada elemento

### Propiedades de los elementos: 

|MA|ME|MI|
|:-:|:-:|:-:|
|$$
\begin{align}
\\
\end{align} 
\begin{bmatrix}
A_{0}\\A_{1}\\A_{2}\\.\\A_{n}\end{bmatrix}\, $$|$$
\begin{align}
\\
\end{align} 
\begin{bmatrix}
E_{0}\\E_{1}\\E_{2}\\.\\E_{n}\end{bmatrix}\, $$|$$
\begin{align}
\\
\end{align} 
\begin{bmatrix}
I_{0}\\I_{1}\\I_{2}\\.\\I_{n}\end{bmatrix}\, $$
                


In [5]:
MA=[[0.12],[0.12],[0.12]] #Matriz con las áreas de la sección transversal de cada elemento (en m2).
ME=[[20000000],[20000000],[20000000]] #Matriz con el módulo de elasticidad de cada elemento (en kPa).
MI=[[0.0016],[0.0016],[0.0016]] #Matriz con las inercias de la sección transversal de cada elemento (en m4).

### Coordenadas de los nodos iniciales y finales de los elementos:

$$
\begin{align}
MC = \\
\end{align} 
\begin{bmatrix}
xi_{0} &yi_{0} &xf_{0} &yf_{0}\\
xi_{1} &yi_{1} &xf_{1} &yf_{1}\\
xi_{2} &yi_{2} &xf_{2} &yf_{2}\\
. &. &. &.\\
xi_{n} &yi_{n} &xf_{n} &yf_{n}\end{bmatrix}\, .$$


In [6]:
MC=[[0,0,0,3],[0,3,4,3],[4,3,4,0]] #Matriz que almacena las coordenadas de los nodos iniciales y finales de cada elemento (en m).

### Cargas aplicadas en los nodos: 

In [7]:
N0=[[20],[0],[0],[0],[0],[-15]] #Matriz de fuerzas aplicadas en los grados de libertad libres (en kN,m).

### Reacciones de empotramiento (coordenadas locales):

In [8]:
MLL=np.zeros((Ne,6,1)) #Matriz de almacenamiento de reacciones de empotramiento en coordenadas locales (en kN,m).
MLL[0]=[[0],[6],[4.5],[0],[6],[-4.5]]
MLL[1]=[[0],[50],[33.33],[0],[50],[-33.33]]

### Desplazamientos en los grados de libertad restringidos:

In [9]:
U1=[[0],[0],[0],[0],[0],[0]] #Matriz de desplazamientos en los grados de libertad restringidos (en m).

## CÁLCULO DE REACCIONES Y DESPLAZAMIENTOS DEL PÓRTICO:

<p style='text-align: justify;'> En esta sección se procesan los datos ingresados para calcular las reacciones (fuerzas en los grados de libertad restringidos) y los desplazamientos de los grados de libertad libres.</p>

### Cálculo de longitudes de los elementos:

In [10]:
ML=np.zeros((Ne,1)) #Matriz de almacenamiento de longitudes
for i in range(Ne):
    ML[i,0]=np.sqrt((MC[i][2]-MC[i][0])**2+(MC[i][3]-MC[i][1])**2)

### Cálculo y almacenamiento de la matriz de rigidez local de los elementos:

In [11]:
MAL=np.zeros((Ne,6,6)) #Matriz de almacenamiento de matrices de rigidez en coordenadas locales
for i in range(Ne):
    MAL[i,0,0]=(MA[i][0]*ME[i][0])/(ML[i][0])
    MAL[i,0,3]=-MAL[i,0,0]
    MAL[i,3,0]=-MAL[i,0,0]
    MAL[i,3,3]=MAL[i,0,0]
    MAL[i,1,1]=(12*ME[i][0]*MI[i][0])/(ML[i][0]**3)
    MAL[i,1,4]=-MAL[i,1,1]
    MAL[i,4,1]=-MAL[i,1,1]
    MAL[i,4,4]=MAL[i,1,1]
    MAL[i,1,2]=(6*ME[i][0]*MI[i][0])/(ML[i][0]**2)
    MAL[i,1,5]=MAL[i,1,2]
    MAL[i,2,1]=MAL[i,1,2]
    MAL[i,2,4]=-MAL[i,1,2]
    MAL[i,4,2]=-MAL[i,1,2]
    MAL[i,4,5]=-MAL[i,1,2]
    MAL[i,5,1]=MAL[i,1,2]
    MAL[i,5,4]=-MAL[i,1,2]
    MAL[i,2,2]=(4*ME[i][0]*MI[i][0])/(ML[i][0])
    MAL[i,5,5]=MAL[i,2,2]
    MAL[i,2,5]=(2*ME[i][0]*MI[i][0])/(ML[i][0])
    MAL[i,5,2]=MAL[i,2,5]

<div class="alert alert-warning">

*Visualización de la matriz de rigidez local de un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver la matriz de rigidez local.

</div>


In [12]:
n=0

In [13]:
df=pd.DataFrame(MAL[n])
df

,0,1,2,3,4,5
0,800000.0,0.000000,0.000000,-800000.0,0.000000,0.000000
1,0.0,14222.222222,21333.333333,0.0,-14222.222222,21333.333333
2,0.0,21333.333333,42666.666667,0.0,-21333.333333,21333.333333
3,-800000.0,0.000000,0.000000,800000.0,0.000000,0.000000
4,0.0,-14222.222222,-21333.333333,0.0,14222.222222,-21333.333333
5,0.0,21333.333333,21333.333333,0.0,-21333.333333,42666.666667


### Cálculo de la matriz de transformación:

In [14]:
MAT=np.zeros((Ne,6,6)) #Matriz de almacenamiento de las matrices de transformación
for i in range(Ne):
    MAT[i,0,0]=(MC[i][2]-MC[i][0])/(ML[i][0])
    MAT[i,1,1]=MAT[i,0,0]
    MAT[i,3,3]=MAT[i,0,0]
    MAT[i,4,4]=MAT[i,0,0]
    MAT[i,1,0]=(MC[i][3]-MC[i][1])/(ML[i][0])
    MAT[i,4,3]=MAT[i,1,0]
    MAT[i,0,1]=-MAT[i,1,0]
    MAT[i,3,4]=-MAT[i,1,0]
    MAT[i,2,2]=1
    MAT[i,5,5]=1

<div class="alert alert-warning">

*Visualización de la matriz de transformación de un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver la matriz de transformación.

</div>

In [15]:
n=0

In [16]:
df=pd.DataFrame(MAT[n])
df

,0,1,2,3,4,5
0,0.0,-1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,-1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0


### Cálculo de la matriz de rigidez de cada elemento en coordenadas globales:

In [17]:
MAG=np.zeros((Ne,6,6)) #Matriz de almacenamiento de las matrices de rigidez en coordenadas globales
for i in range(Ne):
    MAG[i]=np.dot(np.dot(MAT[i],MAL[i]),np.transpose(MAT[i]))  

<div class="alert alert-warning">

*Visualización de la matriz de rigidez global de un elemento del pórtico*

Indique el número del elemento (n) para el cual desea ver la matriz de rigidez global.

</div>

In [18]:
n=0

In [19]:
df=pd.DataFrame(MAG[n])
df

,0,1,2,3,4,5
0,14222.222222,0.0,-21333.333333,-14222.222222,0.0,-21333.333333
1,0.000000,800000.0,0.000000,0.000000,-800000.0,0.000000
2,-21333.333333,0.0,42666.666667,21333.333333,0.0,21333.333333
3,-14222.222222,0.0,21333.333333,14222.222222,0.0,21333.333333
4,0.000000,-800000.0,0.000000,0.000000,800000.0,0.000000
5,-21333.333333,0.0,21333.333333,21333.333333,0.0,42666.666667


### Cálculo de la matriz de rigidez de la estructura:

In [20]:
KG=np.zeros((NGL,NGL)) #Matriz de rigidez global de la estructura
for i in range(Ne):
    for j in range(6):
        for k in range(6):
            KG[int(MGL[i][k]),int(MGL[i][j])]=KG[int(MGL[i][k])][int(MGL[i][j])]+MAG[i][j][k]

In [21]:
df=pd.DataFrame(KG)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,614222.222222,0.0,21333.333333,-600000.000000,0.0,0.000000,-14222.222222,0.0,21333.333333,0.000000,0.0,0.000000
1,0.000000,806000.0,12000.000000,0.000000,-6000.0,12000.000000,0.000000,-800000.0,0.000000,0.000000,0.0,0.000000
2,21333.333333,12000.0,74666.666667,0.000000,-12000.0,16000.000000,-21333.333333,0.0,21333.333333,0.000000,0.0,0.000000
3,-600000.000000,0.0,0.000000,614222.222222,0.0,21333.333333,0.000000,0.0,0.000000,-14222.222222,0.0,21333.333333
4,0.000000,-6000.0,-12000.000000,0.000000,806000.0,-12000.000000,0.000000,0.0,0.000000,0.000000,-800000.0,0.000000
5,0.000000,12000.0,16000.000000,21333.333333,-12000.0,74666.666667,0.000000,0.0,0.000000,-21333.333333,0.0,21333.333333
6,-14222.222222,0.0,-21333.333333,0.000000,0.0,0.000000,14222.222222,0.0,-21333.333333,0.000000,0.0,0.000000
7,0.000000,-800000.0,0.000000,0.000000,0.0,0.000000,0.000000,800000.0,0.000000,0.000000,0.0,0.000000
8,21333.333333,0.0,21333.333333,0.000000,0.0,0.000000,-21333.333333,0.0,42666.666667,0.000000,0.0,0.000000
9,0.000000,0.0,0.000000,-14222.222222,0.0,-21333.333333,0.000000,0.0,0.000000,14222.222222,0.0,-21333.333333


### Cálculo de cargas aplicadas en las luces en coordenadas globales: 

In [22]:
MLG=np.zeros((Ne,6,1)) #Matriz de almacenamiento de reacciones de empotramiento en coordenadas globales
for i in range(Ne):
    MLG[i]=np.dot(MAT[i],MLL[i])

<div class="alert alert-warning">

*Visualización de las reacciones de empotramiento en coordenadas globales de un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver las reacciones de empotramiento en coordenadas globales.

</div>

In [23]:
n=0

In [24]:
print(np.array(MLG[n]))

[[-6. ]
 [ 0. ]
 [ 4.5]
 [-6. ]
 [ 0. ]
 [-4.5]]


### Ensamble de *L* de la estructura: 

In [45]:
L=np.zeros((NGL,1)) 
for i in range(Ne):
    for j in range(6):
        L[MGL[i][j],0]=L[MGL[i][j],0]+MLG[i][j][0]

### Subdivisión de *L*

In [26]:
L0 = L[0:NGLL]
L1 = L[NGLL:NGL]

### Ensamble de *F0* 

In [27]:
F0=N0-L0

### Subdivisión de la matriz de rigidez global de la estructura:

In [28]:
K0 = KG[0:NGLL,0:NGLL]
K1 = KG[0:NGLL,NGLL:NGL]
K2 = KG[NGLL:NGL,0:NGLL]
K3 = KG[NGLL:NGL,NGLL:NGL]  

### Cálculo de desplazamientos:


In [29]:
Feff=F0-np.dot(K1,U1)
U0=np.dot((np.linalg.inv(K0)),Feff) #Desplazamientos en los grados de libertad libres (en m y rad).    
print(np.array(U0))

[[ 1.57250266e-03]
 [-4.98781732e-05]
 [-8.35473002e-04]
 [ 1.53673776e-03]
 [-7.51218268e-05]
 [-1.86039468e-05]]


### Cálculo de reacciones:

In [30]:
F1=np.dot(K2,U0)+np.dot(K3,U1) 
N1=F1+L1 #Reacciones (grados de libertad restringidos)
print(np.array(N1))

[[-10.54105827]
 [ 39.90253853]
 [ 20.22329943]
 [-21.45894173]
 [ 60.09746147]
 [ 32.38685469]]


## CÁLCULO DE LAS FUERZAS INTERNAS DE LA ESTRUCTURA:

<p style='text-align: justify;'> En esta sección se calcula la fuerza interna de cada elemento del pórtico.</p>

### Cálculo de las fuerzas de cada elemento en coodenadas globales:



In [31]:
U=np.concatenate((U0,U1)) #Matriz de desplazamientos de todos los grados de libertad
MU=np.zeros((Ne,6,1)) #Matriz de almacenamiento de vectores de desplazamiento de cada elemento
for i in range(Ne):
    for j in range(6):
        MU[i,j,0]=U[int(MGL[i][j])][0]

<div class="alert alert-warning">

*Visualización de los desplazamientos de un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver los desplazamientos.

</div>

In [32]:
n=0

In [33]:
print(np.array(MU[n]))

[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.57250266e-03]
 [-4.98781732e-05]
 [-8.35473002e-04]]


In [34]:
MFG=np.zeros((Ne,6,1)) #Matriz de almacenamiento de las fuerzas de cada elemento en coordenadas globales
for i in range(Ne):
    MFG[i]=np.dot(MAG[i],MU[i])

<div class="alert alert-warning">

*Visualización de las fuerzas en coordenadas globales de un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver las fuerzas en coordenadas globales.

</div>

In [35]:
n=0

In [36]:
print(np.array(MFG[0]))

[[ -4.54105827]
 [ 39.90253853]
 [ 15.72329943]
 [  4.54105827]
 [-39.90253853]
 [ -2.10012462]]


### Cálculo de las fuerzas en los extremos de cada elemento, *N*

In [37]:
MN=np.zeros((Ne,6,1)) #Matriz de almacenamiento de N de cada elemento
for i in range(Ne):
    MN[i]=MLG[i]+MFG[i]

<div class="alert alert-warning">

*Visualización de N para un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver las fuerzas en los extremos en coordenadas globales.

</div>

In [38]:
n=0

In [39]:
print(np.array(MN[n]))

[[-10.54105827]
 [ 39.90253853]
 [ 20.22329943]
 [ -1.45894173]
 [-39.90253853]
 [ -6.60012462]]


### Cálculo de las fuerzas de cada elemento en coordenadas locales:

In [40]:
MFL=np.zeros((Ne,6,1)) #Matriz de almacenamiento de las fuerzas de cada elemento en coordenadas locales
for i in range(Ne):
    MFL[i]=np.dot(np.transpose(MAT[i]),MN[i])

<div class="alert alert-warning">

*Visualización de las fuerzas en coordenadas locales de un elemento del pórtico:*

Indique el número del elemento (n) para el cual desea ver las fuerzas en coordenadas locales.

</div>

In [41]:
n=0

In [42]:
print(np.array(MFL[n]))

[[ 39.90253853]
 [ 10.54105827]
 [ 20.22329943]
 [-39.90253853]
 [  1.45894173]
 [ -6.60012462]]
